# SFM PIPELINE

In [1]:
%reload_ext autoreload
%autoreload 2
from sfm import SFM
import matplotlib.pyplot as plt
# CHANGE TO IMAGE SOURCE FILE (and db path for pycolmap)
img_src = ""
db_path = "database.db"

## 1. FEATURE EXTRACTION AND MATCHING:
Flag options for BRISK-SIFT/SIFT-SIFT and image preprocessing (for obj. isolation)\

BRISK-SIFT (`brisk=True`)
- recommended when object has lots of texture

SIFT-SIFT (`brisk=False`)
- recommended when object has low texture; able to detect more 'subtle' features at the expense of computation

In [2]:
sfm = SFM(img_src, db_path=db_path, preprocess=True, brisk=True)
# sfm.set_pre_proc_params(dog_flag=True,acceptance_threshold=.9) # USE TO SET PREPROCESS SETTINGS

sfm.ft_extract()
sfm.ft_match()

IMG_4164.png:   2%|▏         | 3/141 [00:00<00:05, 24.92it/s]

.DS_Store is not an img...



IMG_4305.png: 100%|██████████| 141/141 [00:07<00:00, 17.97it/s]




FEATURE EXTRACTION....


Image 140: 100%|██████████| 140/140 [00:33<00:00,  4.13it/s]


...

(40, 62):  29%|██▉       | 40/139 [00:32<01:14,  1.32it/s]                          

RANSAC FAILED, MAY CONTAIN OUTLIERS


(138, 139): 100%|██████████| 139/139 [00:45<00:00,  3.02it/s]                         


Elapsed Time: 45.986788272857666




defaultdict(list,
            {(0, 1): [< cv2.DMatch 0x29459b5b0>,
              < cv2.DMatch 0x29459ab90>,
              < cv2.DMatch 0x29459b6b0>,
              < cv2.DMatch 0x29459b050>,
              < cv2.DMatch 0x29459b410>,
              < cv2.DMatch 0x2dfcabd50>,
              < cv2.DMatch 0x2dfca9e10>,
              < cv2.DMatch 0x2dfca9e90>,
              < cv2.DMatch 0x2dfca9f90>,
              < cv2.DMatch 0x2dfcab3d0>,
              < cv2.DMatch 0x2dfcab390>,
              < cv2.DMatch 0x2dfca9250>,
              < cv2.DMatch 0x2dfcaad90>,
              < cv2.DMatch 0x2dfca9110>,
              < cv2.DMatch 0x2dfca90d0>,
              < cv2.DMatch 0x2dfca9090>,
              < cv2.DMatch 0x2dfca9010>,
              < cv2.DMatch 0x2dfca8f30>,
              < cv2.DMatch 0x2dfcaadb0>,
              < cv2.DMatch 0x2dfcaa4d0>,
              < cv2.DMatch 0x2dfcaa3f0>,
              < cv2.DMatch 0x2dfcabfd0>,
              < cv2.DMatch 0x2dfcabf90>,
              < cv2.DMatch 0x2d

In [ ]:
# VIEW ADJACENCY LIST OF MATCHES:
adj, oadj, _ = sfm.adj_list()
plt.imshow(adj)
# plt.show()
# plt.imshow(oadj)
# plt.show()

## 2. SFM cont'd (3 options)
(Triangulation, Bundle Adjustment, Sparse/Dense Reconstruction)
1. COLMAP GUI
2. PyCOLMAP
3. PyCOLMAP and COLMAP GUI 

**note:** PyCOLMAP (on ft matches) is very inconsistent

### 2.1 via COLMAP GUI **recommended**
Create a colmap project in `img_src` and import features + feature matches. Continue the reconstruction pipeline from there\

ie. import features (via img path) &rarr; import matches (via match.txt for inlier feature matches) &rarr; start reconstruction &rarr; \
dense: undistort &rarr; stereo &rarr; fusion

In [3]:
# create .txt files for image features + matches
sfm.output()
# import these into COLMAP

### 2.2 via PyCOLMAP (**PYCOLMAP MUST BE COMPILED WITH CUDA**, ie. built from source)

In [ ]:
# SET DESTINATION FOLDER AND OBJECT NAME
dest = "output" 
obj_name = "object"
sfm.reconstruction_sparse(dest=dest, obj_name=obj_name)
sfm.reconstruction_dense(dest=dest, model_folder='/0') # can also change model_folder to '\1'


### 2.3a via PyCOLMAP database (contains features + cams only) and COLMAP
**recommended**

In [ ]:
dest = "output" 
obj_name = "object"
sfm.add_cams()
sfm.add_ft()
sfm.output()
# Use .db file + match.txt (inlier matches) in COLMAP GUI

### 2.3b via PyCOLMAP (sparse reconstruction) and COLMAP (dense reconstruction)
Inconsistent for some datasets

In [ ]:
dest = "output" 
obj_name = "object"
sfm.reconstruction_sparse(dest=dest, obj_name=obj_name)
# Use .db file in COLMAP GUI for dense reconstruction

# Surface/Mesh Reconstruction
Use resulting dense point cloud .ply files in the [surface reconstruction pipeline](reconstruction_pipeline.ipynb)